In [1]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import json
import requests
from tools import get_files_from_dir, download_if_new
from corona import calc_effects, vacs, vac_list, vac_list_cum, population
from assumptions import week_delta

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
current_folder = os.getcwd()
data_folder = os.path.join(current_folder,'corona_data')
try:
    os.mkdir(data_folder)
except:
    print('folder already exists')

folder already exists


### External links

In [5]:
url_vaccinations = 'https://impfdashboard.de/static/data/germany_vaccinations_timeseries_v2.tsv'
url_deliveries = 'https://www.bundesgesundheitsministerium.de/fileadmin/Dateien/3_Downloads/C/Coronavirus/Impfstoff/Lieferprognosen_aller_Hersteller_2._Quartal.pdf'

### Downloading files
(including check if the files are new, if not they are deleted directly)

In [6]:
file_path_vaccinations = download_if_new(url_vaccinations,data_folder)
file_path_deliveries = download_if_new(url_deliveries,data_folder)

The-file germany_vaccinations_timeseries_v2.tsv is updated - the update file is added to the data-folder
The-file Lieferprognosen_aller_Hersteller_2._Quartal.pdf is not updated


### Preparing csv-data - Vaccinations already done

In [7]:
df = pd.read_csv(file_path_vaccinations,sep='\t')
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
df.rename(columns={'dosen_biontech_erst_kumulativ':'biontech_1_cum','dosen_biontech_zweit_kumulativ':'biontech_2_cum'
                  ,'dosen_moderna_erst_kumulativ':'moderna_1_cum','dosen_moderna_zweit_kumulativ':'moderna_2_cum'
                  ,'dosen_astrazeneca_erst_kumulativ':'astrazeneca_1_cum','dosen_astrazeneca_zweit_kumulativ':'astrazeneca_2_cum'
                  ,'dosen_johnson_kumulativ':'johnson_1_cum'
                  ,'personen_erst_kumulativ': 'all_1','personen_voll_kumulativ': 'all_full'},inplace=True)
columns_begin = ['date','biontech_1_cum','biontech_2_cum','moderna_1_cum','moderna_2_cum','astrazeneca_1_cum','astrazeneca_2_cum'
                 ,'johnson_1_cum','all_1','all_full']
df['week']= df['date'].apply(lambda x: x.isocalendar()[1])
df['weekday']= df['date'].apply(lambda x: x.isocalendar()[2])

df_total = df[[c for c in columns_begin] + [c for c in df if c not in columns_begin]]
df_total.iloc[-5:]

,date,biontech_1_cum,biontech_2_cum,moderna_1_cum,moderna_2_cum,astrazeneca_1_cum,astrazeneca_2_cum,johnson_1_cum,all_1,all_full,dosen_kumulativ,dosen_differenz_zum_vortag,dosen_erst_differenz_zum_vortag,dosen_zweit_differenz_zum_vortag,dosen_biontech_kumulativ,dosen_moderna_kumulativ,dosen_astrazeneca_kumulativ,impf_quote_erst,impf_quote_voll,indikation_alter_dosen,indikation_beruf_dosen,indikation_medizinisch_dosen,indikation_pflegeheim_dosen,indikation_alter_erst,indikation_beruf_erst,indikation_medizinisch_erst,indikation_pflegeheim_erst,indikation_alter_voll,indikation_beruf_voll,indikation_medizinisch_voll,indikation_pflegeheim_voll,dosen_dim_kumulativ,dosen_kbv_kumulativ,week,weekday
153,2021-05-29,23958706,12398814,2832950,1046159,8482960,664658,425342,35699958,14534973,49809589,408395,155268,253127,36357520,3879109,9147618,0.429,0.175,7644926,5826686,1124640,1761928,4910112,4346229,944820,1001588,2734814,1480457,179820,760340,35110214,14699375,21,6
154,2021-05-30,24020464,12534719,2860877,1065409,8494558,681776,435839,35811738,14717743,50093642,284053,101283,182770,36555183,3926286,9176334,0.431,0.177,7644926,5826686,1124640,1761928,4910112,4346229,944820,1001588,2734814,1480457,179820,760340,35372449,14721193,21,7
155,2021-05-31,24131116,12783285,2887221,1103197,8533635,709416,477711,36029683,15073609,50625581,531939,176073,355866,36914401,3990418,9243051,0.433,0.181,7644926,5826686,1124640,1761928,4910112,4346229,944820,1001588,2734814,1480457,179820,760340,35746619,14878962,22,1
156,2021-06-01,24446340,13185731,2915349,1148038,8588237,741889,561433,36511359,15637091,51587017,961436,397954,563482,37632071,4063387,9330126,0.439,0.188,7644926,5826686,1124640,1761928,4910112,4346229,944820,1001588,2734814,1480457,179820,760340,36117582,15469435,22,2
157,2021-06-02,24890949,13698210,2939086,1182246,8631559,775047,662672,37124266,16318175,52779769,1192752,511668,681084,38589159,4121332,9406606,0.446,0.196,7644926,5826686,1124640,1761928,4910112,4346229,944820,1001588,2734814,1480457,179820,760340,36470056,16309713,22,3


In [8]:
df = df_total[['date','week','weekday']+vac_list_cum[:-1]+['all_1','all_full']].copy()
df = calc_effects(df,shift = 14)

print('Doses last day:', df['date'].iloc[-1].strftime('%d.%m.%y'),round(df_total['dosen_differenz_zum_vortag'].iloc[-1]/1000000,3))
print('Doses 7 days before:', df['date'].iloc[-8].strftime('%d.%m.%y'),round(df_total['dosen_differenz_zum_vortag'].iloc[-8]/1000000,3))
print('Doses 14 days before:', df['date'].iloc[-15].strftime('%d.%m.%y'),round(df_total['dosen_differenz_zum_vortag'].iloc[-15]/1000000,3))
print('Total doses:', round(df_total['dosen_kumulativ'].iloc[-1]/1000000,3))

print('1st dose: ',round(df['all_1'].iloc[-1]/1000000,3),'mln')
print('Full dose: ',round(df['all_full'].iloc[-1]/1000000,3),'mln')
print('1st dose working: ',round(df['all_1'].iloc[-15]/1000000,3),'mln')
print('Full dose working: ',round(df['all_full'].iloc[-15]/1000000,3),'mln')
print('1st dose last week: ',round((df['all_1'].iloc[-1] - df['all_1'].iloc[-8])/1000000,3),'mln')
print('Full dose last week: ',round((df['all_full'].iloc[-1] - df['all_full'].iloc[-8])/1000000,3),'mln')

print('Doses last week: ',round((df_total['dosen_kumulativ'].iloc[-1] - df_total['dosen_kumulativ'].iloc[-8])/1000000,3),'mln')
#print('Doses last week: ',round((df_total['dosen_kumulativ'].iloc[-1] - df_total['dosen_kumulativ'].iloc[-8])/1000000,3),'mln')
print('Doses 2nd-last week: ',round((df_total['dosen_kumulativ'].iloc[-8] 
                                     - df_total['dosen_kumulativ'].iloc[-15])/1000000,3),'mln')

print()

rel_net_effect = round(df['net_effect'].iloc[-1] / df['all_1'].iloc[-1],3)
print(f'The current net effect per person with current 1st dose is {rel_net_effect}')
rel_net_effect_2nd = round(df['net_effect_2nd'].iloc[-1] / df['all_full'].iloc[-1],3)
print(f'The current net effect per person with current 2nd dose is {rel_net_effect_2nd}')
current_real_net = round(df['rel_net_effect'].iloc[-1],3)
print(f'The current relative net effect on the population is ',current_real_net)
R_to_1 = round(1 / (1-current_real_net),3)
print(f'A R-rate of {R_to_1} is reduced to one due to the vaccination')
real_net_14 = round(df['rel_net_effect_after_14'].iloc[-1],3)
print(f'The relative net effect in 14 days on the population is ',real_net_14)
R_to_1_14 = round(1 / (1-real_net_14),3)
print(f'In 14 days a R-rate of {R_to_1_14} is reduced to 1.0 due to the vaccination')
                                            
df.iloc[-10:,:]

Doses last day: 02.06.21 1.193
Doses 7 days before: 26.05.21 1.167
Doses 14 days before: 19.05.21 1.096
Total doses: 52.78
1st dose:  37.124 mln
Full dose:  16.318 mln
1st dose working:  32.363 mln
Full dose working:  10.527 mln
1st dose last week:  2.523 mln
Full dose last week:  3.191 mln
Doses last week:  5.289 mln
Doses 2nd-last week:  4.686 mln

The current net effect per person with current 1st dose is 0.733
The current net effect per person with current 2nd dose is 0.889
The current relative net effect on the population is  0.327
A R-rate of 1.486 is reduced to one due to the vaccination
The relative net effect in 14 days on the population is  0.38
In 14 days a R-rate of 1.613 is reduced to 1.0 due to the vaccination


,date,week,weekday,biontech_1_cum,biontech_2_cum,moderna_1_cum,moderna_2_cum,astrazeneca_1_cum,astrazeneca_2_cum,johnson_1_cum,all_1,all_full,johnson_2_cum,net_effect,net_effect_2nd,rel_net_effect,net_effect_after_14,rel_net_effect_after_14,rel_biontech_14,rel_moderna_14,rel_astrazeneca_14,rel_johnson_14,rel_all_1,rel_all_full,R_to_1,R_to_1_14
148,2021-05-24,21,1,22948877,10496680,2592838,897458,8083643,518352,137272,33762630,12049762,137272,23311620.83,10826634.03,0.280525,28487267.96,0.342807,0.708480,0.071588,0.216752,0.003180,0.406289,0.145003,1.389902,1.521623
149,2021-05-25,21,2,23091164,10799098,2644305,925082,8162490,541750,160653,34058612,12426583,160653,23987398.92,11164462.32,0.288657,28762949.60,0.346125,0.707112,0.072395,0.216807,0.003686,0.409851,0.149538,1.405792,1.529343
150,2021-05-26,21,3,23404255,11365296,2695264,953247,8264008,571327,237706,34601233,13127576,237706,24876029.42,11769520.98,0.299351,29265785.79,0.352176,0.706251,0.072610,0.215778,0.005361,0.416381,0.157973,1.427247,1.543628
151,2021-05-27,21,4,23660676,11840245,2745355,988928,8358788,606074,316665,35081484,13751912,316665,25109931.93,12283724.37,0.302165,29708008.89,0.357497,0.704894,0.073010,0.215061,0.007035,0.422160,0.165486,1.433004,1.556413
152,2021-05-28,21,5,23876655,12218856,2795142,1021633,8447551,643271,398086,35517434,14281846,398086,25561536.37,12728034.43,0.307600,30100787.81,0.362224,0.703214,0.073487,0.214570,0.008729,0.427406,0.171863,1.444251,1.567948
153,2021-05-29,21,6,23958706,12398814,2832950,1046159,8482960,664658,425342,35699958,14534973,425342,25770460.27,12948305.55,0.310114,30270244.29,0.364263,0.702273,0.074152,0.214301,0.009274,0.429602,0.174909,1.449515,1.572977
154,2021-05-30,21,7,24020464,12534719,2860877,1065409,8494558,681776,435839,35811738,14717743,435839,25914734.85,13108662.15,0.311850,30379684.48,0.365580,0.701992,0.074687,0.213852,0.009469,0.430948,0.177109,1.453172,1.576242
155,2021-05-31,22,1,24131116,12783285,2887221,1103197,8533635,709416,477711,36029683,15073609,477711,26220251.32,13413951.26,0.315526,30589612.94,0.368106,0.701197,0.075086,0.213410,0.010307,0.433570,0.181391,1.460977,1.582544
156,2021-06-01,22,2,24446340,13185731,2915349,1148038,8588237,741889,561433,36511359,15637091,561433,26678845.14,13902108.38,0.321045,31030016.87,0.373406,0.701261,0.075016,0.211781,0.011942,0.439367,0.188172,1.472852,1.595929
157,2021-06-02,22,3,24890949,13698210,2939086,1182246,8631559,775047,662672,37124266,16318175,662672,27201474.93,14508091.01,0.327334,31588118.36,0.380122,0.702522,0.074488,0.209145,0.013846,0.446742,0.196368,1.486622,1.613220


### Analysis - how much time is in average between the first and the second dose per vaccine

In [9]:
average_for_days = 31

for vac in ['biontech','moderna','astrazeneca']:
    print(vac)
    df1 = df[['date',vac+'_1_cum']].rename(columns={vac+'_1_cum':vac+'_n'})
    df2 = df[['date',vac+'_2_cum']].rename(columns={vac+'_2_cum':vac+'_n', 'date':'date_2nd'})
    vac_max = df2[vac+'_n'].max()
    print('2nd dose:', round(vac_max/1000000,3))
    dfn = pd.merge_asof(df1[(df1[vac+'_n'] <= vac_max)],df2, on=vac+'_n')
    dfn['2nd_days'] = dfn['date_2nd']-dfn['date']
    average_time_to_2nd_dose = (dfn.iloc[-1-average_for_days:-1]['2nd_days'].mean())#.total_days()
    print(f'average_time_to_2nd_dose last {average_for_days} days       ', average_time_to_2nd_dose)
    average_time_to_2nd_dose_last_14 = (dfn.iloc[-15:-1]['2nd_days'].mean())
    print('average_time_to_2nd_dose_last_14     ', average_time_to_2nd_dose_last_14)
    print(dfn.iloc[-1:,:])    
    print()

biontech
2nd dose: 13.698
average_time_to_2nd_dose last 31 days        34 days 22:27:05.806451
average_time_to_2nd_dose_last_14      36 days 15:25:42.857142
          date  biontech_n   date_2nd 2nd_days
120 2021-04-26    13257976 2021-06-01  36 days

moderna
2nd dose: 1.182
average_time_to_2nd_dose last 31 days        36 days 18:34:50.322580
average_time_to_2nd_dose_last_14      36 days 00:00:00
          date  moderna_n   date_2nd 2nd_days
120 2021-04-26    1153365 2021-06-01  36 days

astrazeneca
2nd dose: 0.775
average_time_to_2nd_dose last 31 days        59 days 15:29:01.935483
average_time_to_2nd_dose_last_14      79 days 05:08:34.285714
         date  astrazeneca_n   date_2nd 2nd_days
66 2021-03-03         695208 2021-05-30  88 days



### Read pdf - planned deliveries

In [10]:
from tabula import read_pdf
df_pdf = read_pdf(file_path_deliveries, pages='all')

In [11]:
df_pdf[0].columns = ['NaN','week','biontech','moderna','astra','johnson']
df_deliveries = df_pdf[0][['week','biontech','moderna','astra','johnson']]

In [12]:
#df_deliveries = df_pdf[0][['Woche', 'BioNTech', 'Moderna', 'AstraZeneca','Johnson &']].copy()
#df_deliveries.rename(columns={'Woche':'week', 'BioNTech':'biontech', 'Moderna':'moderna', 'AstraZeneca':'astra','Johnson &':'johnson'},inplace='True')
df_deliveries = df_deliveries[(df_deliveries['week'].str.slice(0,2)=='KW')].fillna(0)
for col in df_deliveries.columns[1:]:
    df_deliveries[col] = np.where(df_deliveries[col] == '-', 0, df_deliveries[col])
    df_deliveries[col] = df_deliveries[col].str.replace('.','').fillna(0)
    df_deliveries[col] = df_deliveries[col].astype(int)
date_end = dt.datetime(2021,1,3)
#df_deliveries['days'] = df_deliveries['week'].str.slice(-2,).astype(int).astype(int)*7
#df_deliveries['week_end'] = df_deliveries['days'].apply(lambda x: date_end + dt.timedelta(days = x))

df_deliveries['week_end'] = df_deliveries['week'].apply(lambda x: date_end + dt.timedelta(days = int(x[-2:])*7))
df_deliveries['week'] = df_deliveries['week'].str.slice(-2).astype(int)
df_deliveries.rename(columns={'astra': 'astrazeneca'},inplace=True)

### Forecast

In [14]:
df_week = df[['date','week']+vac_list_cum][(df['weekday'] == 7)].copy()
week_last = df_week['week'].iloc[-1]

for vac in vac_list_cum:
    df_week[vac[:-4]] = df_week[vac]-df_week[vac].shift(1)
df_week = df_week.merge(df_deliveries,how = 'outer', on = 'week')
df_week['astrazeneca'].iloc[23:27] = df_week['astrazeneca'].iloc[23:27].mean()
df_week['date'] = df_week['week'].apply(lambda x: date_end + dt.timedelta(days = x*7))
df_week = df_week[['date','week']+vac_list+vacs+vac_list_cum]
df_week.iloc[-10:-3]

for i in df_week.index:
    if df_week.iloc[i]['week'] > week_last and df_week.iloc[i]['week'] < 52:
        #print(df_week.iloc[i]['week'])
        for vac in week_delta.keys():
            df_week.at[i,vac + '_2'] = min(df_week.iloc[i-1][vac],df_week.iloc[i-week_delta[vac]][vac + '_1'])
            df_week.at[i,vac + '_1'] = df_week.iloc[i-1][vac]-df_week.at[i,vac + '_2']
            df_week.at[i,'johnson_1'] = df_week.iloc[i-1]['johnson']
            df_week['johnson_2'] = df_week['johnson_1']
        for vac in vac_list:
            df_week.at[i, vac+'_cum'] = df_week.at[i - 1, vac+'_cum'] + df_week.at[i, vac]
df_week = calc_effects(df_week,shift=2)
df_week.iloc[-10:-3]

/home/thomas/python/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,week,biontech_1,biontech_2,moderna_1,moderna_2,astrazeneca_1,astrazeneca_2,johnson_1,johnson_2,biontech,moderna,astrazeneca,johnson,biontech_1_cum,biontech_2_cum,moderna_1_cum,moderna_2_cum,astrazeneca_1_cum,astrazeneca_2_cum,johnson_1_cum,johnson_2_cum,net_effect,net_effect_2nd,rel_net_effect,net_effect_after_14,rel_net_effect_after_14,rel_biontech_14,rel_moderna_14,rel_astrazeneca_14,rel_johnson_14,all_full,all_1,rel_all_1,rel_all_full,R_to_1,R_to_1_14
21,2021-05-23,20,1491049.0,2022371.0,408216.0,181352.0,732539.0,156763.0,74713.0,74713.0,3408210.0,550800.0,465600.0,295150.0,22863087.0,10346088.0,2547435.0,882728.0,8070941.0,509689.0,129898.0,129898.0,22974927.37,10656098.79,0.276473,28346838.46,0.341117,0.708817,0.070692,0.217467,0.003024,11868403.0,33611361.0,0.404469,0.142821,1.382119,1.517721
22,2021-05-30,21,1157377.0,2188631.0,313442.0,182681.0,423617.0,172087.0,305941.0,305941.0,3409380.0,550800.0,0.0,724300.0,24020464.0,12534719.0,2860877.0,1065409.0,8494558.0,681776.0,435839.0,435839.0,25914734.85,13108662.15,0.311850,30379684.48,0.365580,0.701992,0.074687,0.213852,0.009469,14717743.0,35811738.0,0.430948,0.177109,1.453172,1.576242
23,2021-06-06,22,46038.0,3363342.0,269762.0,281038.0,0.0,0.0,724300.0,724300.0,5135130.0,550800.0,610500.0,513600.0,24066502.0,15898061.0,3130639.0,1346447.0,8494558.0,681776.0,1160139.0,1160139.0,28346838.46,16859019.20,0.341117,31591801.00,0.380166,0.690093,0.080023,0.205647,0.024237,19086423.0,36851838.0,0.443464,0.229680,1.517721,1.613335
24,2021-06-13,23,2559663.0,2575467.0,171259.0,379541.0,362589.0,247911.0,513600.0,513600.0,4585230.0,550800.0,610500.0,240000.0,26626165.0,18473528.0,3301898.0,1725988.0,8857147.0,929687.0,1673739.0,1673739.0,30379684.48,20104486.70,0.365580,34869720.22,0.419612,0.695016,0.078659,0.194645,0.031680,22802942.0,40458949.0,0.486871,0.274404,1.576242,1.722984
25,2021-06-20,24,2154446.0,2430784.0,192635.0,358165.0,0.0,610500.0,240000.0,240000.0,4035330.0,585600.0,610500.0,0.0,28780611.0,20904312.0,3494533.0,2084153.0,8857147.0,1540187.0,1913739.0,1913739.0,31591801.00,23478319.04,0.380166,37371783.61,0.449721,0.704212,0.079397,0.182593,0.033797,26442391.0,43046030.0,0.518003,0.318200,1.613335,1.817259
26,2021-06-27,25,2544281.0,1491049.0,177384.0,408216.0,197608.0,412892.0,0.0,0.0,5962320.0,622800.0,610500.0,0.0,31324892.0,22395361.0,3671917.0,2492369.0,9054755.0,1953079.0,1913739.0,1913739.0,34869720.22,25745098.96,0.419612,40053108.80,0.481987,0.713997,0.079730,0.174738,0.031535,28754548.0,45965303.0,0.553132,0.346023,1.722984,1.930453
27,2021-07-04,26,4804943.0,1157377.0,309358.0,313442.0,0.0,610500.0,0.0,0.0,5962320.0,622800.0,0.0,0.0,36129835.0,23552738.0,3981275.0,2805811.0,9054755.0,2563579.0,1913739.0,1913739.0,37371783.61,27819710.67,0.449721,44475162.17,0.535201,0.734979,0.078434,0.158188,0.028399,30835867.0,51079604.0,0.614676,0.371069,1.817259,2.151465


### Overview table about the most important current and future developments

In [15]:
pd.set_option('display.float_format', '{:.4f}'.format)
df_week[['date','week','rel_net_effect','rel_net_effect_after_14','rel_biontech_14','rel_all_1','rel_all_full','R_to_1','R_to_1_14']].iloc[-15:-3]

,date,week,rel_net_effect,rel_net_effect_after_14,rel_biontech_14,rel_all_1,rel_all_full,R_to_1,R_to_1_14
16,2021-04-18,15,0.1093,0.1681,0.6988,0.1996,0.0669,1.1228,1.2021
17,2021-04-25,16,0.1366,0.1983,0.6912,0.2366,0.0721,1.1582,1.2473
18,2021-05-02,17,0.1681,0.2381,0.7189,0.2846,0.0806,1.2021,1.3125
19,2021-05-09,18,0.1983,0.2765,0.7164,0.3304,0.0953,1.2473,1.3821
20,2021-05-16,19,0.2381,0.3118,0.7180,0.3719,0.1135,1.3125,1.4532
21,2021-05-23,20,0.2765,0.3411,0.7088,0.4045,0.1428,1.3821,1.5177
22,2021-05-30,21,0.3118,0.3656,0.7020,0.4309,0.1771,1.4532,1.5762
23,2021-06-06,22,0.3411,0.3802,0.6901,0.4435,0.2297,1.5177,1.6133
24,2021-06-13,23,0.3656,0.4196,0.6950,0.4869,0.2744,1.5762,1.7230
25,2021-06-20,24,0.3802,0.4497,0.7042,0.5180,0.3182,1.6133,1.8173


### Relevant information for beginning of July - the time for which a useful delivery forecast is available 

In [16]:
df_week_3 = df_week[:-3].copy()
print('date: ',df_week_3.date.iloc[-1])
print('1st dose: ',round(df_week_3['all_1'].iloc[-1]/1000000,3),'mln')
print('Full dose: ',round(df_week_3['all_full'].iloc[-1]/1000000,3),'mln')
print('1st dose working: ',round(df_week_3['all_1'].iloc[-3]/1000000,3),'mln')
print('Full dose working: ',round(df_week_3['all_full'].iloc[-3]/1000000,3),'mln')

print()

rel_net_effect = round(df_week_3['net_effect'].iloc[-1] / df_week_3['all_1'].iloc[-1],3)
print(f'The current net effect per person with current 1st dose is {rel_net_effect}')
rel_net_effect_2nd = round(df_week_3['net_effect_2nd'].iloc[-1] / df_week_3['all_full'].iloc[-1],3)
print(f'The current net effect per person with current 2nd dose is {rel_net_effect_2nd}')
current_real_net = round(df_week_3['rel_net_effect'].iloc[-1],3)
print(f'The current relative net effect on the population is ',current_real_net)
R_to_1 = round(1 / (1-current_real_net),3)
print(f'A R-rate of {R_to_1} is reduced to one due to the vaccination')
real_net_14 = round(df_week_3['rel_net_effect_after_14'].iloc[-1],3)
date_14 = df_week_3.date.iloc[-1] + dt.timedelta(days = 14)
print(f'The relative net effect in 14 days ({date_14}) on the population is ',real_net_14)
R_to_1_14 = round(1 / (1-real_net_14),3)
print(f'In 14 days at the {date_14} a R-rate of {R_to_1_14} is reduced to 1.0 due to the vaccination')

date:  2021-07-04 00:00:00
1st dose:  51.08 mln
Full dose:  30.836 mln
1st dose working:  43.046 mln
Full dose working:  26.442 mln

The current net effect per person with current 1st dose is 0.732
The current net effect per person with current 2nd dose is 0.902
The current relative net effect on the population is  0.45
A R-rate of 1.818 is reduced to one due to the vaccination
The relative net effect in 14 days (2021-07-18 00:00:00) on the population is  0.535
In 14 days at the 2021-07-18 00:00:00 a R-rate of 2.151 is reduced to 1.0 due to the vaccination
